# Module 1：深度學習的數學基礎

## 學習目標

本模組不是要教你「完整的數學」，而是教你「剛好夠用來理解深度學習」的數學。

1. **線性代數**：向量、矩陣、線性變換的直覺
2. **微積分**：偏導數、鏈鎖律（backprop 的數學基礎）
3. **機率**：隨機變數、期望值、常態分佈、最大似然估計

每個概念都會搭配 PyTorch 程式碼，讓你「看得到」數學在做什麼。

---

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# 設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 讓圖片更好看
plt.rcParams['figure.figsize'] = (10, 4)
plt.rcParams['font.size'] = 12

---

## Part 1：線性代數

### 1.1 向量（Vector）

**直覺：** 向量就是「一組有序的數字」，可以想像成空間中的一個點或一個箭頭。

在深度學習中，向量無處不在：
- 一個樣本的特徵 = 一個向量
- 神經網路某一層的輸出 = 一個向量
- Word embedding = 一個向量

In [ ]:
# 向量基本操作
v1 = torch.tensor([1.0, 2.0, 3.0])
v2 = torch.tensor([4.0, 5.0, 6.0])

print("向量 v1:", v1)
print("向量 v2:", v2)

# 向量加法（逐元素相加）
print("\n向量加法 v1 + v2:", v1 + v2)

# 純量乘法（每個元素乘以同一個數）
print("純量乘法 3 * v1:", 3 * v1)

# 逐元素乘法（Hadamard product）
print("逐元素乘法 v1 * v2:", v1 * v2)

### 1.2 內積（Dot Product）

**公式：** $\mathbf{a} \cdot \mathbf{b} = \sum_i a_i b_i = a_1 b_1 + a_2 b_2 + ...$

**直覺：** 內積衡量「兩個向量有多相似」或「在同一個方向上走了多遠」。

- 內積 > 0：兩向量方向相近
- 內積 = 0：兩向量垂直（正交）
- 內積 < 0：兩向量方向相反

**在 DL 中的應用：**
- 神經網路的每一層基本上都是「輸入向量」和「權重向量」做內積
- Attention 機制中，Query 和 Key 做內積來計算相似度

In [ ]:
# 內積
v1 = torch.tensor([1.0, 2.0, 3.0])
v2 = torch.tensor([4.0, 5.0, 6.0])

# 方法 1：torch.dot
dot_product = torch.dot(v1, v2)
print(f"v1 · v2 = {dot_product}")

# 方法 2：手動計算
manual_dot = (v1 * v2).sum()
print(f"手動計算: {manual_dot}")

# 驗證：1*4 + 2*5 + 3*6 = 4 + 10 + 18 = 32
print(f"驗算: 1*4 + 2*5 + 3*6 = {1*4 + 2*5 + 3*6}")

In [ ]:
# 內積的幾何意義：衡量相似度
# 先將向量正規化（變成單位向量），內積就變成 cosine similarity

def cosine_similarity(a, b):
    """計算兩向量的 cosine 相似度"""
    return torch.dot(a, b) / (torch.norm(a) * torch.norm(b))

# 相同方向
a = torch.tensor([1.0, 0.0])
b = torch.tensor([2.0, 0.0])
print(f"相同方向: cos_sim = {cosine_similarity(a, b):.2f}")

# 垂直
a = torch.tensor([1.0, 0.0])
b = torch.tensor([0.0, 1.0])
print(f"垂直: cos_sim = {cosine_similarity(a, b):.2f}")

# 相反方向
a = torch.tensor([1.0, 0.0])
b = torch.tensor([-1.0, 0.0])
print(f"相反方向: cos_sim = {cosine_similarity(a, b):.2f}")

# 45 度角
a = torch.tensor([1.0, 0.0])
b = torch.tensor([1.0, 1.0])
print(f"45度角: cos_sim = {cosine_similarity(a, b):.2f}")

### 1.3 矩陣（Matrix）

**直覺：** 矩陣就是「把多個向量排在一起」，形成一個 2D 的數字表格。

在深度學習中：
- 一批資料 = 一個矩陣（每行是一個樣本）
- 神經網路的權重 = 矩陣
- 圖片 = 矩陣（灰階）或多個矩陣（RGB）

In [ ]:
# 矩陣基本操作
A = torch.tensor([[1., 2.], 
                  [3., 4.], 
                  [5., 6.]])  # 3x2 矩陣

B = torch.tensor([[7., 8.], 
                  [9., 10.], 
                  [11., 12.]])  # 3x2 矩陣

print(f"A shape: {A.shape}")
print(f"A:\n{A}")

# 矩陣加法（shape 必須相同）
print(f"\nA + B:\n{A + B}")

# 轉置（transpose）：行列互換
print(f"\nA 的轉置 A.T shape: {A.T.shape}")
print(f"A.T:\n{A.T}")

### 1.4 矩陣乘法（Matrix Multiplication）

**規則：** 如果 A 是 (m, n) 矩陣，B 是 (n, p) 矩陣，則 A @ B 是 (m, p) 矩陣。

**直覺：** 矩陣乘法可以想成「A 的每一行」和「B 的每一列」做內積。

**在 DL 中的應用：** 神經網路的核心運算就是矩陣乘法！
- 輸入 X: (batch_size, input_dim)
- 權重 W: (input_dim, output_dim)
- 輸出 Y = X @ W: (batch_size, output_dim)

In [ ]:
# 矩陣乘法
A = torch.tensor([[1., 2., 3.],   # 2x3
                  [4., 5., 6.]])

B = torch.tensor([[7., 8.],       # 3x2
                  [9., 10.],
                  [11., 12.]])

C = A @ B  # 或 torch.matmul(A, B)

print(f"A shape: {A.shape}")
print(f"B shape: {B.shape}")
print(f"C = A @ B shape: {C.shape}")
print(f"\nC:\n{C}")

# 驗算 C[0,0]：A 的第 0 行和 B 的第 0 列做內積
# 1*7 + 2*9 + 3*11 = 7 + 18 + 33 = 58
print(f"\n驗算 C[0,0] = 1*7 + 2*9 + 3*11 = {1*7 + 2*9 + 3*11}")

In [ ]:
# 神經網路實例：線性層就是矩陣乘法
batch_size = 4
input_dim = 3
output_dim = 2

# 輸入：4 個樣本，每個 3 維
X = torch.randn(batch_size, input_dim)

# 權重：3 -> 2
W = torch.randn(input_dim, output_dim)
b = torch.randn(output_dim)

# 線性變換：Y = XW + b
Y = X @ W + b

print(f"X shape: {X.shape}")
print(f"W shape: {W.shape}")
print(f"b shape: {b.shape}")
print(f"Y shape: {Y.shape}")

# 這就是 nn.Linear 做的事情！
linear = nn.Linear(input_dim, output_dim)
Y_linear = linear(X)
print(f"\nnn.Linear output shape: {Y_linear.shape}")

### 1.5 線性變換的直覺

**核心概念：** 矩陣乘法 = 線性變換 = 空間的「旋轉、縮放、剪切」

當你把一個向量 $\mathbf{x}$ 乘上矩陣 $A$，得到 $A\mathbf{x}$，就是把 $\mathbf{x}$ 「變換」到另一個位置。

In [ ]:
# 視覺化線性變換

def plot_transformation(A, title):
    """視覺化 2D 線性變換"""
    # 原始的單位正方形四個頂點
    square = torch.tensor([[0., 0.], [1., 0.], [1., 1.], [0., 1.], [0., 0.]]).T
    
    # 變換後的點
    transformed = A @ square
    
    plt.figure(figsize=(8, 4))
    
    # 原始正方形
    plt.subplot(1, 2, 1)
    plt.plot(square[0], square[1], 'b-', linewidth=2, label='Original')
    plt.scatter([0], [0], c='red', s=100, zorder=5)  # 原點
    plt.xlim(-3, 3)
    plt.ylim(-3, 3)
    plt.grid(True)
    plt.axis('equal')
    plt.title('Original')
    
    # 變換後
    plt.subplot(1, 2, 2)
    plt.plot(transformed[0], transformed[1], 'r-', linewidth=2, label='Transformed')
    plt.scatter([0], [0], c='red', s=100, zorder=5)
    plt.xlim(-3, 3)
    plt.ylim(-3, 3)
    plt.grid(True)
    plt.axis('equal')
    plt.title(f'After: {title}')
    
    plt.tight_layout()
    plt.show()

# 縮放矩陣
scale_matrix = torch.tensor([[2., 0.], 
                              [0., 0.5]])
plot_transformation(scale_matrix, 'Scale (2x, 0.5y)')

# 旋轉矩陣（45度）
theta = torch.tensor(np.pi / 4)  # 45 degrees
rotation_matrix = torch.tensor([[torch.cos(theta), -torch.sin(theta)],
                                 [torch.sin(theta), torch.cos(theta)]])
plot_transformation(rotation_matrix, 'Rotate 45°')

# 剪切矩陣
shear_matrix = torch.tensor([[1., 0.5], 
                              [0., 1.]])
plot_transformation(shear_matrix, 'Shear')

### 1.6 Broadcasting（廣播機制）

**問題：** 如果兩個 tensor 的 shape 不同，怎麼做運算？

**答案：** PyTorch 會自動「廣播」較小的 tensor，讓它們 shape 相容。

這在深度學習中超常用，例如：加 bias（向量加到矩陣每一行）

In [ ]:
# Broadcasting 範例

# 矩陣 + 純量：純量會廣播到每個元素
A = torch.tensor([[1., 2.], [3., 4.]])
print(f"A + 10:\n{A + 10}")

# 矩陣 + 向量（行向量）：向量會廣播到每一行
A = torch.tensor([[1., 2., 3.],
                  [4., 5., 6.]])  # shape: (2, 3)
b = torch.tensor([10., 20., 30.])  # shape: (3,)

print(f"\nA shape: {A.shape}")
print(f"b shape: {b.shape}")
print(f"A + b:\n{A + b}")
print("(b 被廣播到 A 的每一行)")

In [ ]:
# 這就是為什麼 Y = XW + b 可以運作
# X @ W 的結果是 (batch, output_dim)
# b 是 (output_dim,)
# 相加時 b 會廣播到每個樣本

batch_size = 4
X = torch.randn(batch_size, 3)
W = torch.randn(3, 2)
b = torch.randn(2)

Y = X @ W + b

print(f"X @ W shape: {(X @ W).shape}")
print(f"b shape: {b.shape}")
print(f"Y = X @ W + b shape: {Y.shape}")
print("\nb 被廣播到 batch 中的每一個樣本！")

---

## Part 2：微積分（Calculus for Backpropagation）

### 2.1 為什麼需要微積分？

**核心問題：** 我們想讓 loss 變小，但參數該往哪個方向調？

**答案：** 用「梯度」（gradient）—— 它告訴你「如果參數稍微變大，loss 會變大還是變小」。

- 梯度為正：增加參數會讓 loss 變大 → 應該減小參數
- 梯度為負：增加參數會讓 loss 變小 → 應該增加參數
- 梯度為零：已經在極值點（最小或最大）

### 2.2 導數（Derivative）的直覺

**定義：** $\frac{df}{dx} = \lim_{h \to 0} \frac{f(x+h) - f(x)}{h}$

**直覺：** 導數就是「斜率」—— 函數在某一點的變化率。

常用導數公式：
- $\frac{d}{dx}(x^n) = n \cdot x^{n-1}$
- $\frac{d}{dx}(e^x) = e^x$
- $\frac{d}{dx}(\ln x) = \frac{1}{x}$

In [ ]:
# 用 PyTorch 自動計算導數

# f(x) = x^2，導數應該是 2x
x = torch.tensor([3.0], requires_grad=True)
y = x ** 2  # y = 9
y.backward()

print(f"f(x) = x^2")
print(f"x = {x.item()}")
print(f"f(x) = {y.item()}")
print(f"df/dx = {x.grad.item()} (應該是 2*3 = 6)")

In [ ]:
# 視覺化導數 = 切線斜率

def f(x):
    return x ** 2

x_vals = torch.linspace(-3, 3, 100)
y_vals = f(x_vals)

# 在 x=1 點的切線
x0 = 1.0
y0 = f(torch.tensor(x0))
slope = 2 * x0  # 導數 = 2x

# 切線方程：y - y0 = slope * (x - x0)
tangent_x = torch.linspace(-1, 3, 50)
tangent_y = y0 + slope * (tangent_x - x0)

plt.figure(figsize=(8, 6))
plt.plot(x_vals, y_vals, 'b-', linewidth=2, label='$f(x) = x^2$')
plt.plot(tangent_x, tangent_y, 'r--', linewidth=2, label=f'Tangent at x={x0} (slope={slope})')
plt.scatter([x0], [y0], c='red', s=100, zorder=5)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Derivative = Slope of Tangent Line')
plt.legend()
plt.grid(True)
plt.xlim(-3, 3)
plt.ylim(-1, 9)
plt.show()

### 2.3 偏導數（Partial Derivative）

**問題：** 如果函數有多個變數怎麼辦？例如 $f(x, y) = x^2 + y^2$

**答案：** 對每個變數分別求導，其他變數當作常數。

$\frac{\partial f}{\partial x} = 2x$（把 y 當常數）

$\frac{\partial f}{\partial y} = 2y$（把 x 當常數）

**梯度（Gradient）** = 所有偏導數組成的向量 = $\nabla f = [\frac{\partial f}{\partial x}, \frac{\partial f}{\partial y}]$

In [ ]:
# 偏導數範例
# f(x, y) = x^2 + y^2

x = torch.tensor([3.0], requires_grad=True)
y = torch.tensor([4.0], requires_grad=True)

f = x**2 + y**2  # f = 9 + 16 = 25
f.backward()

print(f"f(x, y) = x^2 + y^2")
print(f"x = {x.item()}, y = {y.item()}")
print(f"f(x, y) = {f.item()}")
print(f"\n∂f/∂x = {x.grad.item()} (應該是 2*3 = 6)")
print(f"∂f/∂y = {y.grad.item()} (應該是 2*4 = 8)")
print(f"\n梯度 ∇f = [{x.grad.item()}, {y.grad.item()}]")

### 2.4 鏈鎖律（Chain Rule）—— Backpropagation 的數學基礎

**問題：** 如果 $y = f(g(x))$（複合函數），怎麼求 $\frac{dy}{dx}$？

**鏈鎖律：** $\frac{dy}{dx} = \frac{dy}{dg} \cdot \frac{dg}{dx}$

**直覺：** 把微分「串起來」。如果 x 變一點點，g 會變多少？g 變了之後，y 又會變多少？

**在神經網路中：**
```
input → layer1 → layer2 → layer3 → loss
              ↓
        d(loss)/d(input) = d(loss)/d(layer3) × d(layer3)/d(layer2) × d(layer2)/d(layer1) × d(layer1)/d(input)
```

這就是 **Backpropagation**：從 loss 開始，沿著網路「反向」乘回去。

In [ ]:
# 鏈鎖律範例
# y = (2x + 1)^3
# 令 u = 2x + 1，則 y = u^3
# dy/dx = dy/du * du/dx = 3u^2 * 2 = 6(2x+1)^2

x = torch.tensor([2.0], requires_grad=True)

u = 2*x + 1  # u = 5
y = u ** 3   # y = 125

y.backward()

print(f"y = (2x + 1)^3")
print(f"x = {x.item()}")
print(f"y = {y.item()}")
print(f"\ndy/dx = {x.grad.item()}")
print(f"手算驗證: 6 * (2*2+1)^2 = 6 * 25 = {6 * 25}")

In [ ]:
# 神經網路中的鏈鎖律
# 簡化例子：x -> linear -> relu -> linear -> loss

torch.manual_seed(42)

# 資料
x = torch.tensor([[1.0, 2.0]], requires_grad=False)
target = torch.tensor([[1.0]])

# 兩層網路
w1 = torch.randn(2, 4, requires_grad=True)
w2 = torch.randn(4, 1, requires_grad=True)

# Forward pass（記錄計算圖）
h = x @ w1              # 第一層線性
h_relu = torch.relu(h)  # ReLU 激活
y = h_relu @ w2         # 第二層線性
loss = (y - target) ** 2  # MSE loss

print("Forward pass:")
print(f"  x shape: {x.shape}")
print(f"  h = x @ w1 shape: {h.shape}")
print(f"  h_relu shape: {h_relu.shape}")
print(f"  y = h_relu @ w2 shape: {y.shape}")
print(f"  loss: {loss.item():.4f}")

# Backward pass（自動應用鏈鎖律）
loss.backward()

print(f"\nBackward pass (gradients):")
print(f"  w1.grad shape: {w1.grad.shape}")
print(f"  w2.grad shape: {w2.grad.shape}")

### 2.5 梯度下降（Gradient Descent）視覺化

**核心思想：** 沿著梯度的反方向走，就能讓 loss 下降。

$\theta_{new} = \theta_{old} - \eta \cdot \nabla L$

其中 $\eta$ 是學習率。

In [ ]:
# 視覺化梯度下降
# 目標：找 f(x) = x^2 - 4x + 5 的最小值（應該在 x=2）

def f(x):
    return x**2 - 4*x + 5

def grad_f(x):
    return 2*x - 4  # 導數

# 梯度下降
x = 0.0  # 起始點
lr = 0.1  # 學習率
history = [(x, f(x))]

for i in range(20):
    grad = grad_f(x)
    x = x - lr * grad  # 梯度下降更新
    history.append((x, f(x)))

# 視覺化
x_vals = np.linspace(-1, 5, 100)
y_vals = x_vals**2 - 4*x_vals + 5

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(x_vals, y_vals, 'b-', linewidth=2)
history_x = [h[0] for h in history]
history_y = [h[1] for h in history]
plt.plot(history_x, history_y, 'ro-', markersize=8, alpha=0.7)
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('Gradient Descent Path')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history_y, 'g-o')
plt.xlabel('Iteration')
plt.ylabel('f(x)')
plt.title('Loss Curve')
plt.grid(True)

plt.tight_layout()
plt.show()

print(f"最終 x = {history_x[-1]:.4f} (理論最小值在 x = 2)")
print(f"最終 f(x) = {history_y[-1]:.4f} (理論最小值 = 1)")

---

## Part 3：機率與統計

### 3.1 為什麼深度學習需要機率？

深度學習中很多地方都有機率的影子：

1. **不確定性**：模型輸出的是「機率」，不是絕對答案
2. **損失函數**：Cross-entropy loss 來自機率論
3. **正則化**：Dropout 是隨機的
4. **生成模型**：VAE、Diffusion 都是機率模型
5. **訓練資料**：SGD 的「S」= Stochastic（隨機）

### 3.2 隨機變數、期望值、變異數

**隨機變數（Random Variable）：** 一個可能取不同值的變數，每個值有對應的機率。

**期望值（Expected Value）：** 「平均」會得到什麼值
$$E[X] = \sum_x x \cdot P(X=x)$$

**變異數（Variance）：** 值有多「分散」
$$Var(X) = E[(X - E[X])^2] = E[X^2] - E[X]^2$$

**標準差（Standard Deviation）：** $\sigma = \sqrt{Var(X)}$

In [ ]:
# 期望值和變異數

# 擲骰子：X = {1, 2, 3, 4, 5, 6}，每個機率 = 1/6
# E[X] = (1+2+3+4+5+6)/6 = 3.5

# 用 PyTorch 模擬
torch.manual_seed(42)
n_samples = 100000

# 模擬擲骰子
dice_rolls = torch.randint(1, 7, (n_samples,)).float()

mean = dice_rolls.mean()
var = dice_rolls.var()
std = dice_rolls.std()

print(f"擲 {n_samples} 次骰子：")
print(f"  期望值 E[X] = {mean:.4f} (理論值 = 3.5)")
print(f"  變異數 Var[X] = {var:.4f} (理論值 ≈ 2.917)")
print(f"  標準差 σ = {std:.4f}")

### 3.3 常態分佈（Gaussian / Normal Distribution）

**為什麼重要？** 深度學習中到處都是常態分佈：
- 權重初始化：通常用常態分佈
- Batch Normalization：把資料正規化成近似常態
- VAE 的 latent space：假設是常態分佈

**公式：** $p(x) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(x-\mu)^2}{2\sigma^2}\right)$

- $\mu$：平均值（分佈的中心）
- $\sigma$：標準差（分佈的寬度）

In [ ]:
# 常態分佈視覺化

# 標準常態分佈 N(0, 1)
samples_standard = torch.randn(10000)

# N(5, 2) - 平均值 5，標準差 2
samples_shifted = torch.randn(10000) * 2 + 5

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(samples_standard.numpy(), bins=50, density=True, alpha=0.7)
plt.title('Standard Normal N(0, 1)')
plt.xlabel('x')
plt.ylabel('Density')

plt.subplot(1, 2, 2)
plt.hist(samples_shifted.numpy(), bins=50, density=True, alpha=0.7, color='orange')
plt.title('Normal N(5, 2)')
plt.xlabel('x')
plt.ylabel('Density')

plt.tight_layout()
plt.show()

print(f"N(0,1): mean={samples_standard.mean():.3f}, std={samples_standard.std():.3f}")
print(f"N(5,2): mean={samples_shifted.mean():.3f}, std={samples_shifted.std():.3f}")

### 3.4 Softmax 與機率

**問題：** 分類模型最後一層輸出的是「任意數值」（logits），怎麼變成「機率」？

**Softmax：** 把任意數值轉成機率分佈（所有值在 0-1 之間，加總等於 1）

$$softmax(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}}$$

In [ ]:
# Softmax 範例

# 模型輸出的 logits（原始分數）
logits = torch.tensor([2.0, 1.0, 0.1])

# 手動計算 softmax
exp_logits = torch.exp(logits)
softmax_manual = exp_logits / exp_logits.sum()

# 用 PyTorch 的 softmax
softmax_torch = torch.softmax(logits, dim=0)

print(f"Logits: {logits}")
print(f"\nSoftmax (手動): {softmax_manual}")
print(f"Softmax (PyTorch): {softmax_torch}")
print(f"\n機率總和: {softmax_torch.sum():.4f}")

# 視覺化
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.bar(['Class 0', 'Class 1', 'Class 2'], logits.numpy())
plt.title('Logits (Raw Scores)')
plt.ylabel('Score')

plt.subplot(1, 2, 2)
plt.bar(['Class 0', 'Class 1', 'Class 2'], softmax_torch.numpy())
plt.title('After Softmax (Probabilities)')
plt.ylabel('Probability')
plt.ylim(0, 1)

plt.tight_layout()
plt.show()

### 3.5 Cross-Entropy Loss（交叉熵損失）

**直覺：** 衡量「預測的機率分佈」和「真實的機率分佈」有多不同。

**公式（分類任務）：**
$$L = -\sum_i y_i \log(p_i)$$

其中 $y_i$ 是真實標籤（one-hot），$p_i$ 是預測機率。

如果只有一個正確類別 $c$，簡化成：$L = -\log(p_c)$

**直覺：**
- 如果預測正確類別的機率接近 1，$-\log(1) \approx 0$，loss 很小
- 如果預測正確類別的機率接近 0，$-\log(0) \to \infty$，loss 很大

In [ ]:
# Cross-Entropy Loss 範例

# 假設 3 個類別，真實標籤是 class 0
target = torch.tensor([0])  # class index

# 情況 1：模型很有信心，預測正確
logits_good = torch.tensor([[5.0, 1.0, 0.1]])  # class 0 分數最高

# 情況 2：模型不確定
logits_uncertain = torch.tensor([[1.0, 0.9, 0.8]])  # 分數差不多

# 情況 3：模型很有信心，但預測錯誤
logits_bad = torch.tensor([[0.1, 5.0, 1.0]])  # class 1 分數最高

criterion = nn.CrossEntropyLoss()

loss_good = criterion(logits_good, target)
loss_uncertain = criterion(logits_uncertain, target)
loss_bad = criterion(logits_bad, target)

print("真實標籤: Class 0\n")
print(f"情況 1 (預測正確，高信心): logits={logits_good[0].tolist()}")
print(f"  probs={torch.softmax(logits_good, dim=1)[0].tolist()}")
print(f"  loss={loss_good.item():.4f}\n")

print(f"情況 2 (不確定): logits={logits_uncertain[0].tolist()}")
print(f"  probs={torch.softmax(logits_uncertain, dim=1)[0].tolist()}")
print(f"  loss={loss_uncertain.item():.4f}\n")

print(f"情況 3 (預測錯誤，高信心): logits={logits_bad[0].tolist()}")
print(f"  probs={torch.softmax(logits_bad, dim=1)[0].tolist()}")
print(f"  loss={loss_bad.item():.4f}")

### 3.6 最大似然估計（Maximum Likelihood Estimation, MLE）

**核心思想：** 找到讓「觀察到的資料最有可能發生」的參數。

**直覺：**
1. 你有一些觀察到的資料
2. 假設資料來自某個機率分佈（有未知參數）
3. 找到讓這些資料「最有可能」被生成的參數

**和 Loss 的關係：**
- 最大化 likelihood = 最小化 negative log-likelihood
- Cross-entropy loss 其實就是 negative log-likelihood！

In [ ]:
# MLE 範例：估計常態分佈的參數
# 假設資料來自 N(μ, σ)，我們要找出 μ 和 σ

torch.manual_seed(42)

# 生成資料（真實 μ=5, σ=2）
true_mu = 5.0
true_sigma = 2.0
data = torch.randn(1000) * true_sigma + true_mu

# MLE 估計（對常態分佈，MLE 就是樣本均值和樣本標準差）
estimated_mu = data.mean()
estimated_sigma = data.std()

print(f"真實參數: μ={true_mu}, σ={true_sigma}")
print(f"MLE 估計: μ={estimated_mu:.4f}, σ={estimated_sigma:.4f}")

# 視覺化
plt.figure(figsize=(8, 4))
plt.hist(data.numpy(), bins=50, density=True, alpha=0.7, label='Data')

x = torch.linspace(-5, 15, 200)
# 標準常態分佈的 PDF
pdf = torch.exp(-0.5 * ((x - estimated_mu) / estimated_sigma) ** 2) / (estimated_sigma * np.sqrt(2 * np.pi))
plt.plot(x, pdf, 'r-', linewidth=2, label=f'Fitted N({estimated_mu:.2f}, {estimated_sigma:.2f})')

plt.xlabel('x')
plt.ylabel('Density')
plt.title('MLE: Fitting a Normal Distribution')
plt.legend()
plt.show()

---

## 練習題（已完成，請閱讀理解）

### 練習 1：向量運算練習

**目標：** 熟悉向量內積和 cosine similarity

**Hint：**
- 內積：`torch.dot(a, b)` 或 `(a * b).sum()`
- 向量長度：`torch.norm(a)`
- Cosine similarity = 內積 / (長度乘積)

In [ ]:
# 練習 1：計算詞向量相似度
# 假設這是三個詞的 embedding

# 模擬詞向量（實際應用中會用預訓練的 word2vec 或其他）
word_embeddings = {
    'king': torch.tensor([0.5, 0.8, 0.1, 0.3]),
    'queen': torch.tensor([0.6, 0.9, 0.2, 0.4]),
    'apple': torch.tensor([0.1, 0.2, 0.9, 0.8]),
}

def cosine_similarity(a, b):
    """計算 cosine similarity"""
    return torch.dot(a, b) / (torch.norm(a) * torch.norm(b))

# 計算所有詞對的相似度
words = list(word_embeddings.keys())
print("Cosine Similarity Matrix:")
print(f"{'':>8}", end='')
for w in words:
    print(f"{w:>8}", end='')
print()

for w1 in words:
    print(f"{w1:>8}", end='')
    for w2 in words:
        sim = cosine_similarity(word_embeddings[w1], word_embeddings[w2])
        print(f"{sim.item():>8.3f}", end='')
    print()

print("\n觀察：'king' 和 'queen' 很相似，'apple' 和它們都不太像")

### 練習 2：手動實現梯度下降

**目標：** 不用 PyTorch autograd，手動計算梯度並更新參數

**Hint：**
- 對於 $f(x) = x^2$，導數是 $2x$
- 梯度下降更新：$x_{new} = x_{old} - lr \times gradient$

In [ ]:
# 練習 2：手動梯度下降找 f(x) = (x-3)^2 的最小值
# 最小值應該在 x = 3

def f(x):
    """目標函數"""
    return (x - 3) ** 2

def grad_f(x):
    """手動計算的梯度：d/dx (x-3)^2 = 2(x-3)"""
    return 2 * (x - 3)

# 梯度下降
x = 0.0  # 起始點
lr = 0.1  # 學習率
history = []

for i in range(30):
    loss = f(x)
    grad = grad_f(x)
    history.append({'step': i, 'x': x, 'f(x)': loss, 'grad': grad})
    
    # 更新 x
    x = x - lr * grad
    
    if i < 5 or i >= 25:
        print(f"Step {i:2d}: x = {history[-1]['x']:.4f}, f(x) = {history[-1]['f(x)']:.4f}, grad = {history[-1]['grad']:.4f}")
    elif i == 5:
        print("...")

print(f"\n最終 x = {x:.6f} (目標: 3.0)")

In [ ]:
# 對比：用 PyTorch autograd
x = torch.tensor([0.0], requires_grad=True)
lr = 0.1

for i in range(30):
    # Forward
    loss = (x - 3) ** 2
    
    # Backward
    loss.backward()
    
    # Update (不要追蹤這個操作的梯度)
    with torch.no_grad():
        x -= lr * x.grad
    
    # 清零梯度
    x.grad.zero_()

print(f"PyTorch autograd 結果: x = {x.item():.6f}")

### 練習 3：理解 Softmax 的溫度參數

**目標：** 了解 temperature 如何影響 softmax 輸出

**Hint：**
- Softmax with temperature: $softmax(x_i / T)$
- T 越大，分佈越「平坦」（更不確定）
- T 越小，分佈越「尖銳」（更確定）
- T → 0 時，變成 argmax（只有最大的是 1）

In [ ]:
# 練習 3：Softmax 溫度

logits = torch.tensor([2.0, 1.0, 0.5, 0.1])

temperatures = [0.1, 0.5, 1.0, 2.0, 5.0]

plt.figure(figsize=(12, 4))

for i, T in enumerate(temperatures):
    probs = torch.softmax(logits / T, dim=0)
    plt.subplot(1, 5, i+1)
    plt.bar(range(4), probs.numpy())
    plt.title(f'T = {T}')
    plt.ylim(0, 1)
    plt.xticks(range(4))
    if i == 0:
        plt.ylabel('Probability')

plt.suptitle('Softmax with Different Temperatures', y=1.02)
plt.tight_layout()
plt.show()

print("觀察：")
print("- T=0.1 (很小): 幾乎是 one-hot，只有最大的有機率")
print("- T=1.0 (標準): 正常的 softmax")
print("- T=5.0 (很大): 接近均勻分佈")

### 練習 4：用矩陣乘法實現一個簡單的神經網路層

**目標：** 手動實現 forward 和 backward pass

**Hint：**
- Forward: $Y = XW + b$
- Backward: $\frac{\partial L}{\partial W} = X^T \cdot \frac{\partial L}{\partial Y}$

In [ ]:
# 練習 4：手動實現線性層

class ManualLinear:
    """手動實現的線性層（不用 autograd）"""
    
    def __init__(self, in_features, out_features):
        # 初始化權重（用小的隨機數）
        self.W = torch.randn(in_features, out_features) * 0.01
        self.b = torch.zeros(out_features)
        
        # 儲存梯度
        self.W_grad = None
        self.b_grad = None
        
        # 儲存 forward 的輸入（backward 時需要）
        self.X = None
    
    def forward(self, X):
        """Forward pass: Y = XW + b"""
        self.X = X  # 儲存輸入
        return X @ self.W + self.b
    
    def backward(self, grad_output):
        """Backward pass: 計算參數的梯度"""
        # grad_output 是 dL/dY（從後面傳來的梯度）
        
        # dL/dW = X^T @ (dL/dY)
        self.W_grad = self.X.T @ grad_output
        
        # dL/db = sum(dL/dY, dim=0)
        self.b_grad = grad_output.sum(dim=0)
        
        # dL/dX = (dL/dY) @ W^T（傳給前一層）
        grad_input = grad_output @ self.W.T
        
        return grad_input
    
    def update(self, lr):
        """梯度下降更新參數"""
        self.W -= lr * self.W_grad
        self.b -= lr * self.b_grad

# 測試
torch.manual_seed(42)

# 資料
X = torch.randn(4, 3)  # 4 個樣本，3 維特徵
y_true = torch.randn(4, 2)  # 目標

# 手動實現的層
layer = ManualLinear(3, 2)

# Forward
y_pred = layer.forward(X)

# Loss (MSE)
loss = ((y_pred - y_true) ** 2).mean()

# dL/dY for MSE: 2 * (y_pred - y_true) / n
grad_output = 2 * (y_pred - y_true) / y_pred.numel()

# Backward
layer.backward(grad_output)

print(f"Forward output shape: {y_pred.shape}")
print(f"Loss: {loss.item():.4f}")
print(f"\nW_grad shape: {layer.W_grad.shape}")
print(f"b_grad shape: {layer.b_grad.shape}")

In [ ]:
# 驗證：和 PyTorch autograd 比較

# 用相同的初始權重
layer_torch = nn.Linear(3, 2)
with torch.no_grad():
    layer_torch.weight.copy_(layer.W.T)  # nn.Linear 的 weight 是轉置的
    layer_torch.bias.copy_(layer.b)

# Forward
y_pred_torch = layer_torch(X)

# Loss
loss_torch = nn.MSELoss()(y_pred_torch, y_true)

# Backward
loss_torch.backward()

print("PyTorch vs 手動實現的梯度比較：")
print(f"\nW_grad (PyTorch):")
print(layer_torch.weight.grad.T)  # 轉置回來比較
print(f"\nW_grad (手動):")
print(layer.W_grad)
print(f"\n差異: {(layer_torch.weight.grad.T - layer.W_grad).abs().max().item():.6f}")

## Module 1 總結

### 線性代數
- **向量**：一組有序數字，是神經網路的基本資料單位
- **內積**：衡量相似度，神經網路每一層的核心運算
- **矩陣乘法**：線性變換，`Y = XW + b` 是神經網路的基礎
- **Broadcasting**：自動擴展維度，讓不同 shape 的 tensor 可以運算

### 微積分
- **導數**：函數的變化率（斜率）
- **偏導數**：多變數函數對單一變數的導數
- **梯度**：所有偏導數組成的向量，指向函數增長最快的方向
- **鏈鎖律**：複合函數求導，是 backpropagation 的數學基礎

### 機率
- **期望值 & 變異數**：描述分佈的中心和散佈程度
- **常態分佈**：深度學習中最常用的分佈
- **Softmax**：把任意數值轉成機率分佈
- **Cross-Entropy**：衡量預測分佈和真實分佈的差異
- **MLE**：找到最能解釋觀察資料的參數

---

## Part 4：進階數學概念（深入理解）

### 4.1 特徵值與特徵向量（Eigenvalues & Eigenvectors）

**為什麼重要？**
- PCA（主成分分析）的核心
- 理解矩陣的「本質行為」
- 神經網路優化的收斂分析

**定義：** 對於矩陣 $A$，如果 $Av = \lambda v$，則 $v$ 是特徵向量，$\lambda$ 是特徵值。

**直覺：** 特徵向量是矩陣作用下「只會被縮放而不會改變方向」的向量。

In [ ]:
# 特徵值分解範例

# 建立一個對稱矩陣（協方差矩陣通常是對稱的）
A = torch.tensor([[4., 2.],
                  [2., 3.]], dtype=torch.float32)

# 計算特徵值和特徵向量
eigenvalues, eigenvectors = torch.linalg.eigh(A)  # eigh 用於對稱矩陣

print(f"矩陣 A:\n{A}")
print(f"\n特徵值: {eigenvalues}")
print(f"\n特徵向量:\n{eigenvectors}")

# 驗證：A @ v = λ * v
for i in range(2):
    v = eigenvectors[:, i]
    lam = eigenvalues[i]
    Av = A @ v
    lambda_v = lam * v
    print(f"\n驗證特徵向量 {i+1}:")
    print(f"  A @ v = {Av.tolist()}")
    print(f"  λ * v = {lambda_v.tolist()}")
    print(f"  差異: {(Av - lambda_v).abs().max().item():.6f}")

In [ ]:
# PCA（主成分分析）實例：使用特徵值分解

torch.manual_seed(42)

# 生成有相關性的 2D 資料
n_samples = 500
mean = torch.tensor([0., 0.])
# 資料主要沿著 (1, 0.5) 方向分佈
x1 = torch.randn(n_samples)
x2 = 0.5 * x1 + 0.3 * torch.randn(n_samples)  # x2 和 x1 有相關性
data = torch.stack([x1, x2], dim=1)

# 中心化
data_centered = data - data.mean(dim=0)

# 計算協方差矩陣
cov_matrix = (data_centered.T @ data_centered) / (n_samples - 1)

# 特徵值分解
eigenvalues, eigenvectors = torch.linalg.eigh(cov_matrix)

# 按特徵值大小排序（大的在前）
idx = eigenvalues.argsort(descending=True)
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]

print(f"協方差矩陣:\n{cov_matrix}")
print(f"\n特徵值（解釋的變異數）: {eigenvalues}")
print(f"變異數解釋比例: {eigenvalues / eigenvalues.sum()}")
print(f"\n主成分（特徵向量）:\n{eigenvectors}")

In [ ]:
# 視覺化 PCA
plt.figure(figsize=(12, 5))

# 原始資料
plt.subplot(1, 2, 1)
plt.scatter(data[:, 0], data[:, 1], alpha=0.5, s=10)

# 畫主成分方向
origin = data.mean(dim=0)
for i, (eigval, color, label) in enumerate(zip(eigenvalues, ['red', 'blue'], ['PC1', 'PC2'])):
    vec = eigenvectors[:, i] * np.sqrt(eigval) * 2  # 縮放以便視覺化
    plt.arrow(origin[0], origin[1], vec[0], vec[1], 
              head_width=0.1, head_length=0.05, fc=color, ec=color, linewidth=2)
    plt.text(origin[0] + vec[0], origin[1] + vec[1], f' {label}', fontsize=12, color=color)

plt.xlabel('x1')
plt.ylabel('x2')
plt.title('原始資料 + 主成分方向')
plt.axis('equal')
plt.grid(True)

# 投影到主成分空間
plt.subplot(1, 2, 2)
data_pca = data_centered @ eigenvectors  # 投影
plt.scatter(data_pca[:, 0], data_pca[:, 1], alpha=0.5, s=10)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('投影到主成分空間後')
plt.axis('equal')
plt.grid(True)

plt.tight_layout()
plt.show()

print("\n觀察：")
print("- PC1（紅色）沿著資料變異最大的方向")
print("- PC2（藍色）與 PC1 垂直")
print("- 投影後的資料，PC1 和 PC2 不相關了")

### 4.2 數值梯度檢查（Gradient Checking）

**為什麼重要？** 當你自己實現 backward 時，很容易出錯。數值梯度檢查可以驗證你的梯度計算是否正確。

**方法：** 用差分近似導數
$$\frac{\partial f}{\partial \theta} \approx \frac{f(\theta + \epsilon) - f(\theta - \epsilon)}{2\epsilon}$$

如果數值梯度和解析梯度相差很小，說明實現是正確的。

In [ ]:
def numerical_gradient(f, x, epsilon=1e-5):
    """
    用差分法計算數值梯度
    f: 函數，輸入 tensor，輸出純量
    x: 要計算梯度的 tensor
    """
    grad = torch.zeros_like(x)
    x_flat = x.flatten()
    grad_flat = grad.flatten()
    
    for i in range(len(x_flat)):
        # 保存原值
        old_val = x_flat[i].item()
        
        # f(x + epsilon)
        x_flat[i] = old_val + epsilon
        f_plus = f(x.view_as(grad))
        
        # f(x - epsilon)
        x_flat[i] = old_val - epsilon
        f_minus = f(x.view_as(grad))
        
        # 差分
        grad_flat[i] = (f_plus - f_minus) / (2 * epsilon)
        
        # 恢復原值
        x_flat[i] = old_val
    
    return grad

# 測試：簡單函數 f(x) = x^T A x（二次型）
A = torch.tensor([[2., 1.],
                  [1., 3.]], dtype=torch.float32)

def quadratic_form(x):
    return (x @ A @ x).sum()

# 解析梯度：d/dx (x^T A x) = (A + A^T) x = 2Ax（當 A 對稱時）
x = torch.tensor([1., 2.], requires_grad=True)

# PyTorch autograd
y = quadratic_form(x)
y.backward()
analytic_grad = x.grad.clone()

# 數值梯度
x_no_grad = x.detach().clone()
numerical_grad = numerical_gradient(quadratic_form, x_no_grad)

print(f"x = {x.detach().tolist()}")
print(f"f(x) = x^T A x = {y.item()}")
print(f"\n解析梯度 (PyTorch): {analytic_grad.tolist()}")
print(f"數值梯度: {numerical_grad.tolist()}")
print(f"\n相對誤差: {((analytic_grad - numerical_grad).norm() / (analytic_grad.norm() + 1e-8)).item():.2e}")
print("\n（相對誤差 < 1e-5 表示梯度正確）")

### 4.3 資訊理論基礎（Information Theory）

資訊理論在深度學習中非常重要，尤其是：
- Cross-entropy loss 的理論基礎
- VAE 的 ELBO
- Diffusion models 的推導

**熵（Entropy）：** 衡量不確定性
$$H(p) = -\sum_x p(x) \log p(x)$$

**KL 散度（Kullback-Leibler Divergence）：** 衡量兩個分佈的差異
$$D_{KL}(p \| q) = \sum_x p(x) \log \frac{p(x)}{q(x)}$$

**交叉熵（Cross-Entropy）：**
$$H(p, q) = -\sum_x p(x) \log q(x) = H(p) + D_{KL}(p \| q)$$

In [ ]:
# 熵的範例

def entropy(p):
    """計算離散分佈的熵（使用自然對數）"""
    # 避免 log(0)
    p = p[p > 0]
    return -(p * torch.log(p)).sum()

# 均勻分佈（最大熵）
uniform_dist = torch.tensor([0.25, 0.25, 0.25, 0.25])

# 偏斜分佈（較低熵）
skewed_dist = torch.tensor([0.7, 0.1, 0.1, 0.1])

# 確定分佈（最低熵 = 0）
certain_dist = torch.tensor([1.0, 0.0, 0.0, 0.0])

print("熵（Entropy）範例：")
print("-" * 40)
print(f"均勻分佈 {uniform_dist.tolist()}")
print(f"  熵 = {entropy(uniform_dist):.4f}")
print(f"  理論最大熵 = ln(4) = {np.log(4):.4f}")

print(f"\n偏斜分佈 {skewed_dist.tolist()}")
print(f"  熵 = {entropy(skewed_dist):.4f}")

print(f"\n確定分佈 {certain_dist.tolist()}")
print(f"  熵 = {entropy(certain_dist):.4f}")

print("\n直覺：熵越大，不確定性越高；確定的事件熵為 0")

In [ ]:
# KL 散度範例

def kl_divergence(p, q):
    """計算 KL(p || q)"""
    # 只計算 p > 0 的地方
    mask = p > 0
    return (p[mask] * torch.log(p[mask] / q[mask])).sum()

# 真實分佈
p = torch.tensor([0.4, 0.3, 0.2, 0.1])

# 不同的近似分佈
q1 = torch.tensor([0.4, 0.3, 0.2, 0.1])  # 完全相同
q2 = torch.tensor([0.35, 0.30, 0.25, 0.10])  # 稍微不同
q3 = torch.tensor([0.25, 0.25, 0.25, 0.25])  # 均勻分佈
q4 = torch.tensor([0.1, 0.1, 0.1, 0.7])  # 差很多

print("KL 散度（KL Divergence）範例：")
print(f"真實分佈 p = {p.tolist()}")
print("-" * 50)
print(f"q1（相同）= {q1.tolist()}")
print(f"  KL(p||q1) = {kl_divergence(p, q1):.6f}")

print(f"\nq2（稍有不同）= {q2.tolist()}")
print(f"  KL(p||q2) = {kl_divergence(p, q2):.6f}")

print(f"\nq3（均勻）= {q3.tolist()}")
print(f"  KL(p||q3) = {kl_divergence(p, q3):.6f}")

print(f"\nq4（差很多）= {q4.tolist()}")
print(f"  KL(p||q4) = {kl_divergence(p, q4):.6f}")

print("\n注意：")
print("- KL 散度 >= 0，當且僅當 p = q 時等於 0")
print("- KL 散度不對稱：KL(p||q) != KL(q||p)")

In [ ]:
# 高斯分佈的 KL 散度（VAE 的關鍵）
# KL(N(μ₁, σ₁²) || N(μ₂, σ₂²)) 有解析解

def gaussian_kl(mu1, sigma1, mu2, sigma2):
    """
    計算兩個高斯分佈的 KL 散度
    KL(N(μ₁, σ₁²) || N(μ₂, σ₂²))
    """
    var1 = sigma1 ** 2
    var2 = sigma2 ** 2
    
    kl = torch.log(sigma2 / sigma1) + (var1 + (mu1 - mu2)**2) / (2 * var2) - 0.5
    return kl

# VAE 中常用的情況：KL(N(μ, σ²) || N(0, 1))
def gaussian_kl_to_standard_normal(mu, log_var):
    """
    計算 KL(N(μ, exp(log_var)) || N(0, 1))
    這是 VAE 的正則化項
    """
    # 解析公式：-0.5 * (1 + log_var - μ² - exp(log_var))
    return -0.5 * (1 + log_var - mu**2 - torch.exp(log_var))

print("高斯 KL 散度（用於 VAE）：")
print("-" * 50)

# 例子：不同分佈到標準常態的 KL
test_cases = [
    (0.0, 0.0, "N(0, 1) - 標準常態"),
    (1.0, 0.0, "N(1, 1) - 平均值偏移"),
    (0.0, 1.0, "N(0, e) - 變異數較大"),
    (0.0, -1.0, "N(0, 1/e) - 變異數較小"),
    (2.0, 1.0, "N(2, e) - 兩者都偏"),
]

for mu, log_var, desc in test_cases:
    mu_t = torch.tensor([mu])
    log_var_t = torch.tensor([log_var])
    kl = gaussian_kl_to_standard_normal(mu_t, log_var_t)
    print(f"{desc}")
    print(f"  KL = {kl.item():.4f}")

print("\n在 VAE 中，這個 KL 散度作為正則化項，")
print("鼓勵 latent space 接近標準常態分佈 N(0, 1)")

### 4.4 Batch Normalization 的數學

Batch Normalization 是深度學習中最重要的技巧之一。讓我們理解它的數學原理。

**前向傳播：**
1. 計算 mini-batch 的均值和變異數
2. 標準化：$\hat{x} = \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}}$
3. 縮放和平移：$y = \gamma \hat{x} + \beta$

**為什麼有效？**
- 減少 internal covariate shift
- 允許使用更大的學習率
- 有輕微的正則化效果

In [ ]:
# 手動實現 Batch Normalization

class ManualBatchNorm1d:
    """手動實現的 Batch Normalization（用於理解原理）"""
    
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum
        
        # 可學習參數
        self.gamma = torch.ones(num_features)   # 縮放
        self.beta = torch.zeros(num_features)   # 平移
        
        # 運行統計量（用於推理時）
        self.running_mean = torch.zeros(num_features)
        self.running_var = torch.ones(num_features)
        
        self.training = True
    
    def forward(self, x):
        """
        x: shape (batch_size, num_features)
        """
        if self.training:
            # 訓練時：用 batch 統計量
            batch_mean = x.mean(dim=0)
            batch_var = x.var(dim=0, unbiased=False)
            
            # 更新運行統計量
            self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * batch_mean
            self.running_var = (1 - self.momentum) * self.running_var + self.momentum * batch_var
            
            mean, var = batch_mean, batch_var
        else:
            # 推理時：用運行統計量
            mean, var = self.running_mean, self.running_var
        
        # 標準化
        x_norm = (x - mean) / torch.sqrt(var + self.eps)
        
        # 縮放和平移
        out = self.gamma * x_norm + self.beta
        
        return out

# 測試
torch.manual_seed(42)
x = torch.randn(32, 10)  # 32 個樣本，10 個特徵

bn_manual = ManualBatchNorm1d(10)
bn_torch = nn.BatchNorm1d(10)

# 前向傳播
y_manual = bn_manual.forward(x)
y_torch = bn_torch(x)

print("手動 BatchNorm vs PyTorch BatchNorm：")
print(f"輸入 x: mean={x.mean():.4f}, std={x.std():.4f}")
print(f"\n手動 BN 輸出: mean={y_manual.mean():.4f}, std={y_manual.std():.4f}")
print(f"PyTorch BN 輸出: mean={y_torch.mean():.4f}, std={y_torch.std():.4f}")

# 檢查每個特徵是否被正確標準化
print(f"\n每個特徵的統計量（應該接近 mean=0, std=1）:")
print(f"手動 BN - 特徵均值: {y_manual.mean(dim=0)[:5].tolist()}")
print(f"手動 BN - 特徵標準差: {y_manual.std(dim=0)[:5].tolist()}")

---

## 完整總結與實戰 Checklist

### 本 Module 涵蓋的內容：

| 主題 | 重點 | 應用場景 |
|------|------|----------|
| **向量與內積** | 相似度計算 | Attention、Embedding |
| **矩陣乘法** | 線性變換 | 全連接層、所有神經網路層 |
| **Broadcasting** | 維度擴展 | Bias 加法、標準化 |
| **導數與偏導** | 變化率 | 理解梯度含義 |
| **鏈鎖律** | 複合函數求導 | Backpropagation |
| **梯度下降** | 優化方法 | 模型訓練 |
| **機率分佈** | 不確定性建模 | 分類輸出、生成模型 |
| **Softmax** | 轉換為機率 | 分類任務 |
| **Cross-Entropy** | 分佈差異 | 分類 Loss |
| **特徵值分解** | 矩陣本質 | PCA、理解協方差 |
| **數值梯度** | 梯度驗證 | Debug 自定義層 |
| **KL 散度** | 分佈距離 | VAE、正則化 |
| **Batch Norm** | 標準化 | 加速訓練、穩定性 |

### 實戰 Checklist：

- [ ] 能用 PyTorch 計算向量內積和 cosine similarity
- [ ] 理解矩陣乘法的維度規則：`(m, n) @ (n, p) -> (m, p)`
- [ ] 能解釋 backpropagation 的鏈鎖律原理
- [ ] 知道 softmax 的作用和溫度參數的影響
- [ ] 理解 cross-entropy loss 的含義
- [ ] 能用數值梯度檢查自己的梯度實現
- [ ] 理解 KL 散度在 VAE 中的作用
- [ ] 能手動實現 batch normalization

### 下一步：Module 2 - 多層感知機 (MLP) 與訓練技巧